In [1]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt

import glob

import subprocess
from subprocess import call

import re

import pickle

import sys                                                                         
from gffutils.iterators import DataIterator  
from gtfparse import read_gtf

/home/alla/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Refseq v109: find all non-AUG proteoforms, with sole start and additional forms 

* using web-interface, extract genbank file with genes containing non-AUG in their description 
* parse it, get a gene list 

In [2]:
# new_refseq_nonAUG.gb

gene_names_with_non_AUG_starts = []

gb_file = "data/RefSeq_nonAUG_genes_May2021.gb"
for i, gb_record in enumerate(SeqIO.parse(open(gb_file,"r"), "genbank")) :
    # now do something with the record
    
    print ('record %s / 146' % i)
    # print ("Name %s, %i features" % (gb_record.name, len(gb_record.features)))
    for feat in gb_record.features:
        try:
            if feat.type == 'CDS':
                non_AUG = None
                cds_seq = str(feat.extract(gb_record.seq))
                non_AUG_from_cds_seq = cds_seq[0:3] 
                stop_codon = cds_seq[-3:]
                gene_name = feat.qualifiers['gene'][0]
                for el in feat.qualifiers['note']:
                    if el.find('non-AUG') != -1:
                        non_AUG = el.split('non-AUG')[1].split(' ')[1].replace('(', '').replace(')', '')
                #li.append([gb_record.name, gene_name, str(gb_record.seq), non_AUG, cds_seq, non_AUG_from_cds_seq, stop_codon])
                        gene_names_with_non_AUG_starts.append(gene_name)
                
                
        except Exception as e:
            print (gb_record.name, e)


print ('%s genes with non-AUG proteoforms annotated in refseq' % len(set(gene_names_with_non_AUG_starts)))
print (' | '.join(list(set(gene_names_with_non_AUG_starts))))

record 0 / 146
record 1 / 146
record 2 / 146
record 3 / 146
record 4 / 146
record 5 / 146
record 6 / 146
record 7 / 146
record 8 / 146
record 9 / 146
record 10 / 146
record 11 / 146
record 12 / 146
record 13 / 146
record 14 / 146
record 15 / 146
record 16 / 146
record 17 / 146
record 18 / 146
record 19 / 146
record 20 / 146
record 21 / 146
record 22 / 146
record 23 / 146
record 24 / 146
record 25 / 146
record 26 / 146
record 27 / 146
record 28 / 146
record 29 / 146
record 30 / 146
record 31 / 146
record 32 / 146
record 33 / 146
record 34 / 146
record 35 / 146
record 36 / 146
record 37 / 146
record 38 / 146
record 39 / 146
record 40 / 146
record 41 / 146
record 42 / 146
record 43 / 146
record 44 / 146
record 45 / 146
record 46 / 146
record 47 / 146
record 48 / 146
record 49 / 146
record 50 / 146
record 51 / 146
record 52 / 146
record 53 / 146
record 54 / 146
record 55 / 146
record 56 / 146
record 57 / 146
record 58 / 146
record 59 / 146
record 60 / 146
record 61 / 146
record 62 / 146
re

* download ALL genbank files for 43 selected genes (including all proteoforms: AUG and nonAUG)

* we can use following query:

In [3]:
print ('('+'[All Fields] OR '.join(list(set(gene_names_with_non_AUG_starts)))+')')

(TRPV6[All Fields] OR TXNRD3[All Fields] OR BAG1[All Fields] OR HCK[All Fields] OR STIM2[All Fields] OR PTEN[All Fields] OR PIGX[All Fields] OR TEAD1[All Fields] OR PANK2[All Fields] OR DDX17[All Fields] OR LUZP6[All Fields] OR MYC[All Fields] OR FGF2[All Fields] OR WT1[All Fields] OR TEAD4[All Fields] OR MORN2[All Fields] OR NR1I2[All Fields] OR C16orf82[All Fields] OR IRAG1[All Fields] OR KCTD11[All Fields] OR OAZ3[All Fields] OR PIM1[All Fields] OR WDR26[All Fields] OR NPW[All Fields] OR SWI5[All Fields] OR EIF4G2[All Fields] OR SP3[All Fields] OR MDFIC[All Fields] OR HMHB1[All Fields] OR NELFB[All Fields] OR NPTXR[All Fields] OR TEAD3[All Fields] OR POU5F1[All Fields] OR MRTFA[All Fields] OR RARB[All Fields] OR CEBPA[All Fields] OR GTF3A[All Fields] OR PEG10[All Fields] OR PRPS1L1[All Fields] OR R3HCC1[All Fields] OR FNDC5[All Fields] OR CACNG8[All Fields] OR RNF187[All Fields] OR VEGFA)


In [4]:
li3 = []

gb_file = "data/RefSeq_genes_with_AUG_and_nonAUG_May2021.gb"
for gb_record in SeqIO.parse(open(gb_file,"r"), "genbank") :
    # now do something with the record
    # print ("Name %s, %i features" % (gb_record.name, len(gb_record.features)))
    for feat in gb_record.features:
        try:
            if (feat.type == 'CDS') & (gb_record.name.find('NM_') != -1):
                non_AUG = None
                cds_seq = str(feat.extract(gb_record.seq))
                non_AUG_from_cds_seq = cds_seq[0:3] 
                stop_codon = cds_seq[-3:]
                gene_name = feat.qualifiers['gene'][0]
                for el in feat.qualifiers['note']:
                    if el.find('non-AUG') != -1:
                        non_AUG = el.split('non-AUG')[1].split(' ')[1].replace('(', '').replace(')', '')
                li3.append([gb_record.name, gene_name, str(gb_record.seq), non_AUG, cds_seq, non_AUG_from_cds_seq, stop_codon])
                
        except Exception as e:
            print (gb_record.name, e)

NM_004688 'note'
NM_001126049 'note'
NM_003110 'note'
NM_021035 'note'
NM_003215 'note'
NM_024040 'note'
NM_014872 'note'
NM_152441 'note'
NM_020964 'note'
NM_006361 'note'
NM_001142610 'note'
NM_014683 'note'
NM_152657 'note'
NM_138811 'note'
NM_153350 'note'
NM_001291281 'note'
NM_001371351 'note'
NM_001371352 'note'
NM_006255 'note'
NM_022484 'note'
NM_153029 'note'
NM_003577 'note'
NM_015630 'note'
NM_021009 'note'
NM_005842 'note'
NM_001318538 'note'
NM_001318537 'note'
NM_001318536 'note'
NM_001287497 'note'
NM_213598 'note'
NM_018012 'note'
NM_016097 'note'


Check which ones have only non-AUG proteoforms and which ones have both

In [5]:
refseq_df = pd.DataFrame(li3)
refseq_df.columns = ['tr_id', 'gene', 'transcript_seq', 'non_AUG_start', 'cds_seq', 'non_AUG_from_cds_seq', 'stop_codon']

d2 = refseq_df[['tr_id', 'gene', 'non_AUG_from_cds_seq']].groupby('gene')['non_AUG_from_cds_seq'].apply(set).apply(list).apply(', '.join).to_dict()
df = pd.DataFrame(zip(d2.keys(), d2.values()), columns = ['gene', 'codons'])
df = df[df['codons'] != 'ATG']


print ('%s genes have only non-AUG proteoroms' % df[(df['codons'].str.len() == 3) & (df['codons'] != 'ATG')].gene.nunique())
print (', '.join(df[(df['codons'].str.len() == 3) & (df['codons'] != 'ATG')].gene.tolist()))
print ()

print ('%s genes have both AUG and non-AUG proteoroms' % df[df['codons'].str.len() >= 6].gene.nunique())
print (', '.join(df[df['codons'].str.len() >= 6].gene.tolist()))
print ()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


21 genes have only non-AUG proteoroms
C16orf82, CACNG8, DDX17, EIF4G2, GTF3A, HMHB1, LUZP6, MORN2, MYC, NELFB, NPTXR, NPW, PIGX, PRPS1L1, R3HCC1, RNF187, STIM2, TEAD1, TEAD3, TRPV6, TXNRD3

23 genes have both AUG and non-AUG proteoroms
BAG1, CEBPA, FGF2, FNDC5, HCK, IRAG1, KCTD11, MDFIC, MRTFA, NR1I2, OAZ3, PANK2, PEG10, PIM1, POU5F1, PTEN, RARB, SP3, SWI5, TEAD4, VEGFA, WDR26, WT1



In [6]:
refseq_df['non_AUG_start'] = refseq_df['non_AUG_start'].str.replace('U', 'T')
refseq_only_non_AUG  = refseq_df[refseq_df['gene'].isin(df[(df['codons'].str.len() == 3) & (df['codons'] != 'ATG')].gene.unique())][['tr_id', 'gene', 'non_AUG_start']]
refseq_only_non_AUG.to_csv('tmp_res/refseq109_genes_with_sole_non_AUG.txt', sep='\t', index=False)

In [7]:
refseq_both = refseq_df[refseq_df['gene'].isin(df[df['codons'].str.len() >= 6].gene.unique())][['tr_id', 'gene', 'non_AUG_from_cds_seq']]
refseq_both.columns = ['tr_id', 'gene', 'start']
refseq_both.to_csv('tmp_res/refseq109_genes_with_both_AUG_and_non_AUG.txt', sep='\t', index=False)

# Gencode 35

In [8]:
metadata_pc_g35 = pd.read_csv('tmp_res/metadata_pc_g35.txt', sep='\t')

metadata_pc_g35 = metadata_pc_g35.drop('CDS_start', axis=1)

metadata_pc_g25 = pd.read_csv('tmp_res/metadata_pc_g25.txt', sep='\t')


df = read_gtf("data/gencode.v35.annotation.gtf")
df_genes = df[df["feature"] == "gene"]
df_CDS = df[df["feature"] == "CDS"]

annotation_CDS = df_CDS[['transcript_id', 'transcript_type', 'gene_name', 'tag']]
annotation_CDS.columns = ['tr_id', 'transcript_type', 'gene', 'tag']

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [9]:
df.columns

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id',
       'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name',
       'transcript_support_level', 'tag', 'havana_transcript', 'exon_number',
       'exon_id', 'ont', 'protein_id', 'ccdsid'],
      dtype='object')

In [10]:
df[df['transcript_id'] == 'ENST00000594371.1'][['transcript_id', 'feature', 'transcript_type', 'tag']].drop_duplicates()

,transcript_id,feature,transcript_type,tag
375113,ENST00000594371.1,transcript,protein_coding,exp_conf
375114,ENST00000594371.1,exon,protein_coding,exp_conf
375115,ENST00000594371.1,CDS,protein_coding,exp_conf
375126,ENST00000594371.1,stop_codon,protein_coding,exp_conf
375127,ENST00000594371.1,UTR,protein_coding,exp_conf


* non-AUG tag 

In [11]:
# we will create df with transcript type and tags 
li = []
                                                        
for i,feature in enumerate(DataIterator('data/gencode.v35.annotation.gtf')): 
    if i % 1000000 == 0:
        print (i)
    try:
        attr = feature.attributes
        gene_type = attr['gene_type'][0]
        transcript_id = attr['transcript_id'][0]
        gene_name = attr['gene_name'][0]
        tag = attr['tag']
        transcript_type = attr['transcript_type'][0]
            
        # cds_start_NF;cds_end_NF 
        if 'non_ATG_start' in tag:            
            li.append([transcript_id, gene_name, transcript_type, ';'.join(tag), 'non_ATG_start'])
        else:
            li.append([transcript_id, gene_name, transcript_type, ';'.join(tag), None])
            
    except Exception as e:
        continue           
    

0
1000000
2000000


In [12]:
df = pd.DataFrame(li, columns = ['tr_id', 'gene', 'biotype', 'tags', 'non-ATG-tag']).drop_duplicates()

# select genes with complete CDS, protein-coding transcript type and non-AUG tag 
non_AUG_annotated_pc_and_cds_complete = df[(df['non-ATG-tag'] == 'non_ATG_start') & (df['biotype'] == 'protein_coding') & 
                                                          (~df['tags'].str.contains('cds_start_NF')) &
                                                         (~df['tags'].str.contains('cds_end_NF'))]

In [13]:
non_AUG_annotated_pc_and_cds_complete.tr_id.nunique(), non_AUG_annotated_pc_and_cds_complete.gene.nunique()

(67, 46)

In [14]:
non_AUG_annotated_pc_and_cds_complete[non_AUG_annotated_pc_and_cds_complete['gene'] == 'MYC'].to_csv('MYC.txt', sep='\t', index=False)

In [15]:
genes_with_non_AUG_proteoforms = list(non_AUG_annotated_pc_and_cds_complete['gene'].unique())

print (len(genes_with_non_AUG_proteoforms))

print ('TEAD1' in genes_with_non_AUG_proteoforms)

print ('TEAD3' in genes_with_non_AUG_proteoforms)

46
False
True


In [16]:
#genes_with_non_AUG_proteoforms

In [17]:
metadata_pc_g35[metadata_pc_g35['gene'] == 'TEAD1'][['tr_id',  'cds_start_codon', 'cds_stop_codon']].merge(annotation_CDS[annotation_CDS['gene'] == 'TEAD1'].drop_duplicates(), on='tr_id').to_csv('tmp_res/TEAD1.txt', sep='\t', index=False)

In [18]:
metadata_pc_g35[metadata_pc_g35['gene'] == 'MYC'][['tr_id',  'cds_start_codon', 'cds_stop_codon']].merge(annotation_CDS[annotation_CDS['gene'] == 'MYC'].drop_duplicates(), on='tr_id').to_csv('tmp_res/MYC.txt', sep='\t', index=False)

* different approach: get transcripts with full CDS, protein-coding type and extract start and stop as first 3 and last 3 nucleotides of CDS

In [19]:
cds_complete_and_pc_df = df[(df['biotype'] == 'protein_coding') & (~df['tags'].str.contains('cds_start_NF')) & (~df['tags'].str.contains('cds_end_NF'))]

print (cds_complete_and_pc_df.shape, df.shape)

(58397, 5) (168395, 5)


In [20]:
t = metadata_pc_g35[['tr_id', 'cds_seq']].merge(cds_complete_and_pc_df, on=['tr_id'], how='inner')
print (t.shape, metadata_pc_g35.shape, )

li_ = []

for row in t.to_numpy():
    tr_id = row[0]
    gene = row[2]
    cds_seq = row[1]
    tags = row[-2]
    nonAUGtag = row[-1]
    
    start_CDS = cds_seq[0:3]
    stop_CDS = cds_seq[-3:]
    li_.append([tr_id, gene, tags, nonAUGtag, start_CDS, stop_CDS])
    
tmp = pd.DataFrame(li_, columns = ['tr_id', 'gene', 'tags', 'nonAUGtag', 'start_CDS', 'stop_CDS'])

(58397, 6) (101486, 13)


* clearly something's wrong: starts and stops are weird. CDSs must be still truncated regardless of absent tags 

In [21]:
t = pd.DataFrame(tmp.groupby(by='start_CDS').count())[['tr_id']].reset_index().sort_values(by='tr_id', ascending=False)

t.columns = ['First_3nt_of_CDS', '#_transcripts']

t[t['First_3nt_of_CDS'] != 'ATG']['#_transcripts'].sum()

314

In [22]:
# truncated CDS even if there is no such tag in .gtf file 

tmp[(~tmp['gene'].isin(genes_with_non_AUG_proteoforms)) & (tmp['start_CDS'] != 'ATG')].to_csv('tmp_res/gencode35_troubled_genes_with_most_likely_truncated_CDS.txt', sep='\t', index=False)

In [23]:
tmp[(~tmp['gene'].isin(genes_with_non_AUG_proteoforms)) & (tmp['start_CDS'] != 'ATG')]

,tr_id,gene,tags,nonAUGtag,start_CDS,stop_CDS
784,ENST00000622840.1,TEX46,basic,None,AGG,TAA
1857,ENST00000334103.11,AGBL4,basic,None,GAC,TAA
1882,ENST00000612209.1,C1orf185,basic,None,ATA,TGA
2014,ENST00000610401.4,SSBP3,basic,None,GCG,GAG
2119,ENST00000545203.2,ROR1,basic,None,TCT,TAA
...,...,...,...,...,...,...
57645,ENST00000639890.1,RBMX2,basic,None,AGC,TAG
58384,ENST00000361390.2,MT-ND1,basic;appris_principal_1,None,ATA,CTA
58385,ENST00000361453.3,MT-ND2,basic;appris_principal_1,None,ATT,TAT
58391,ENST00000361227.2,MT-ND3,basic;appris_principal_1,None,ATA,AAT


In [24]:
', '.join(list(tmp.stop_CDS.unique())), ', '.join(list(tmp.start_CDS.unique()))

('TAG, TAA, TGA, CGC, ATG, GTG, ACC, AAT, GGC, AGA, GAG, CCC, TCG, AGG, AAA, AAG, TCT, CAG, TGG, ATA, GGT, GAA, CGA, TTT, TTC, TTG, CCT, GCC, TTA, CTG, TAT, GGA, CAC, ACA, CAA, TCA, ATC, TAC, CAT, TGT, GAT, TGC, CCA, GTT, GCG, GAC, AGC, CTT, GCA, CTA, TCC, GGG, CTC, GTA, AGT, GCT, ATT, ACT',
 'ATG, GTG, AGG, ATA, CTG, GAC, GCG, TCT, CCT, GTT, AAC, TTG, ACG, GAT, ATC, ATT, TAA, AGT, TTT, AAA, GGG, GCT, GCA, TTA, GGT, CCC, GTC, CCA, TCA, GAA, GCC, AGC, GTA, AGA, AAG, GGA, ACA, GGC, CAG, CCG, AAT, TCC, ACT, TAC, CTT, CTC, TGC, CAT, TGG, TTC, CTA, TGT, CAA, GAG, ACC')

In [25]:
tmp[tmp['start_CDS'] == 'TGA']

# KCNH7-205 ENST00000621889.1 has GGG as a start of translated sequence in gencode and it still has tags = basic...

# same with PACRG-210 ENST00000611387.1

,tr_id,gene,tags,nonAUGtag,start_CDS,stop_CDS


we have to stick to non-AUG tag... 

In [26]:
# pc transcripts, complete cds -> merge with metadata
pr_trascript_cds_complete_df = cds_complete_and_pc_df.merge(metadata_pc_g35, on=['gene', 'tr_id'], how='inner')

# select genes with non-AUG
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS = pr_trascript_cds_complete_df[pr_trascript_cds_complete_df['gene'].isin(genes_with_non_AUG_proteoforms)]

print (df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS.shape, df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS.gene.nunique(), Counter(df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS.cds_start_codon.tolist()))

(174, 16) 46 Counter({'ATG': 105, 'CTG': 42, 'GTG': 9, 'ACG': 6, 'TTG': 5, 'ATA': 4, 'AAC': 1, 'AAG': 1, 'ATT': 1})


In [27]:
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS[df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS['cds_start_codon'] == 'AAC']

,tr_id,gene,biotype,tags,non-ATG-tag,gene_tr,transcript_seq,5UTR_start_seq,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
31319,ENST00000534810.2,YPEL4,protein_coding,alternative_5_UTR;non_ATG_start;basic,non_ATG_start,YPEL4-210,TTCACCCACAGACCGAGGTACAGAACGAGAGACAACCTCTGCCCCC...,TTCACCCACAGACCGAGGTACAGAACGAGAGACAACCTCTGCCCCC...,AACCTGCTCCGCTGCTTCACCTGCGACCGTCTCTGCGGGGGCTGCA...,AAC,TGA,227,704,0,227,ENST00000534810.2|ENSG00000166793.12|OTTHUMG00...


In [28]:
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS = pr_trascript_cds_complete_df[pr_trascript_cds_complete_df['gene'].isin(genes_with_non_AUG_proteoforms)]

# CDS_start codon 
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS['CDS_start'] = df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS['cds_seq'].str.slice(0, 3)

# gene - start codons dataframe
d1 = df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS.groupby('gene')['CDS_start'].apply(set).apply(list).apply(', '.join).to_dict()
gene_codon_g35_df = pd.DataFrame(zip(d1.keys(), d1.values()), columns = ['gene', 'codons'])

/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [29]:
print ('there are both AUG and non-AUG proteoforms for %s genes' % gene_codon_g35_df[(gene_codon_g35_df['codons'].str.contains('ATG')) & (gene_codon_g35_df['codons'].str.len() >= 6)].shape[0])
print (', '.join(gene_codon_g35_df[(gene_codon_g35_df['codons'].str.contains('ATG')) & (gene_codon_g35_df['codons'].str.len() >= 6)].gene.unique()))
print ()

print ('there are only AUG codon proteoform for %s genes (although its non-AUG annotated)' % gene_codon_g35_df[(gene_codon_g35_df['codons'] == 'ATG')].shape[0])
print (', '.join(gene_codon_g35_df[(gene_codon_g35_df['codons'] == 'ATG')].gene.unique()))
print ()

print ('there are only non-AUG codon proteoform for %s genes' % gene_codon_g35_df[(gene_codon_g35_df['codons'].str.len() <=3) & (gene_codon_g35_df['codons'] != 'ATG')].shape[0])
print (', '.join(gene_codon_g35_df[(gene_codon_g35_df['codons'].str.len() <=3) & (gene_codon_g35_df['codons'] != 'ATG')].gene.unique()))

there are both AUG and non-AUG proteoforms for 30 genes
BAG1, EED, EFCAB14, EIF4G3, FNDC5, HCK, IFT46, KCTD11, MORN2, MRTFA, MYC, NFKBID, NR1I2, OAZ3, PEG10, PIGX, POLG, POU5F1, R3HCC1, SBK1, SLC22A17, SWI5, TEAD4, TRAK1, UFSP1, WDR26, WT1, YPEL1, YPEL2, YPEL4

there are only AUG codon proteoform for 1 genes (although its non-AUG annotated)
CASK

there are only non-AUG codon proteoform for 15 genes
C16orf82, CACNG8, DDX17, EIF4G2, GTF3A, NELFB, NPTXR, NPW, PRPS1L1, RNF187, STIM2, TEAD3, TRPV6, TUG1, TXNRD3


In [30]:
gene_codon_g35_df[(gene_codon_g35_df['codons'] == 'ATG')]

,gene,codons
3,CASK,ATG


In [31]:
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS[df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS['tr_id'] == 'ENST00000378179.9']

,tr_id,gene,biotype,tags,non-ATG-tag,gene_tr,transcript_seq,5UTR_start_seq,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id,CDS_start
56386,ENST00000378179.9,CASK,protein_coding,CAGE_supported_TSS;non_ATG_start;basic,non_ATG_start,CASK-206,TTGTGAGAACATTCCTTCCTAGAAGGAGTTGGGTTTGCCCTTTCCC...,TTGTGAGAACATTCCTTCCTAGAAGGAGTTGGGTTTGCCCTTTCCC...,ATGGCCTTACTTCAGACTCACGACGTAGTGGCACATGAAGTTTACA...,ATG,TAG,646,2047,0,646,ENST00000378179.9|ENSG00000147044.22|OTTHUMG00...,ATG


In [32]:
both_proteoforms_g35_genes = gene_codon_g35_df[(gene_codon_g35_df['codons'].str.contains('ATG')) & (gene_codon_g35_df['codons'].str.len() >= 6)].gene.unique()

only_non_AUG_proteoforms_genes = gene_codon_g35_df[(gene_codon_g35_df['codons'].str.len() <=3) & (gene_codon_g35_df['codons'] != 'ATG')].gene.unique()

print (len(only_non_AUG_proteoforms_genes), len(both_proteoforms_g35_genes))

15 30


In [33]:
both_proteoforms_g35_genes = gene_codon_g35_df[(gene_codon_g35_df['codons'].str.contains('ATG')) & (gene_codon_g35_df['codons'].str.len() >= 6)].gene.unique()

only_non_AUG_proteoforms_genes = gene_codon_g35_df[(gene_codon_g35_df['codons'].str.len() <=3) & (gene_codon_g35_df['codons'] != 'ATG')].gene.unique()

print (len(only_non_AUG_proteoforms_genes), len(both_proteoforms_g35_genes))

15 30


In [34]:
df = df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS.copy()

g35_only_df = df[df['gene'].isin(only_non_AUG_proteoforms_genes)][['tr_id', 'gene', 'cds_start_codon']]



g35_only_df2 = g35_only_df.merge(annotation_CDS, on=['tr_id', 'gene'], how='inner').drop_duplicates()

g35_only_df3 = g35_only_df2[(g35_only_df2['transcript_type'] == 'protein_coding') 
            & (~g35_only_df2['tag'].str.contains('CDS_start_NF')) &
            (~g35_only_df2['tag'].str.contains('CDS_end_NF'))]

print (g35_only_df.gene.nunique(), g35_only_df3.gene.nunique())


g35_only_df3.to_csv('tmp_res/gencode35_only_non_AUG_PC_complete_CDS.txt', sep='\t', index=False)

15 15


In [35]:
g35_both_df = df[df['gene'].isin(both_proteoforms_g35_genes)][['tr_id', 'gene', 'cds_start_codon']]

g35_both_df2 = g35_both_df.merge(annotation_CDS, on=['tr_id', 'gene'], how='inner').drop_duplicates()

g35_both_df3 = g35_both_df2[(g35_both_df2['transcript_type'] == 'protein_coding') 
            & (~g35_both_df2['tag'].str.contains('CDS_start_NF')) &
            (~g35_both_df2['tag'].str.contains('CDS_end_NF'))]

print (g35_both_df.gene.nunique(), g35_both_df3.gene.nunique())

g35_both_df3.to_csv('tmp_res/gencode35_both_AUG_non_AUG_PC_complete_CDS.txt', sep='\t', index=False)

30 30


* ENST00000467011.5 - no non_ATG_start tag in tags (g35), but transcript sequence starts with TTG
* there is ATG protein-coding transcript ENST00000478049.1 which has 'cds_end_NF' tag meaning truncated 3'end of CDS. Can it still be considered as protein-coding? 

# Comparison RefSeq and Gencode35

In [36]:
refseq_only_non_AUG['type_refseq'] = 'only'
refseq_only_non_AUG.columns = ['tr_id_refseq', 'gene', 'start', 'type_refseq']

refseq_both['type_refseq'] = 'both'
refseq_both.columns = ['tr_id_refseq', 'gene', 'start', 'type_refseq']

g35_only_df['type_g35'] = 'only'
g35_only_df.columns = ['tr_id_g35', 'gene', 'start', 'type_g35']

g35_both_df['type_g35'] = 'both'
g35_both_df.columns = ['tr_id_g35', 'gene', 'start', 'type_g35']

In [37]:
# only non-AUG both in g35 and refseq
print ('%s only non-AUG both in g35 and refseq' % len(list(set(refseq_only_non_AUG.gene.unique()).intersection(set(g35_only_df.gene.unique())))))
print (', '.join(list(set(refseq_only_non_AUG.gene.unique()).intersection(set(g35_only_df.gene.unique())))))

print ('%s genes non-AUG in g35' % g35_only_df.gene.nunique())
print ('%s genes non-AUG in refseq' % refseq_only_non_AUG.gene.nunique())

14 only non-AUG both in g35 and refseq
NELFB, TRPV6, TXNRD3, GTF3A, PRPS1L1, STIM2, TEAD3, NPTXR, CACNG8, RNF187, NPW, EIF4G2, DDX17, C16orf82
15 genes non-AUG in g35
21 genes non-AUG in refseq


In [38]:
set(g35_only_df.gene.unique()) - set(refseq_only_non_AUG.gene.unique()).intersection(set(g35_only_df.gene.unique()))

{'TUG1'}

In [39]:
', '.join(list(set(refseq_only_non_AUG.gene.unique()) - set(refseq_only_non_AUG.gene.unique()).intersection(set(g35_only_df.gene.unique()))))

'PIGX, R3HCC1, TEAD1, LUZP6, MYC, MORN2, HMHB1'

In [40]:
g35_only_df[g35_only_df['gene'] == 'TUG1']

,tr_id_g35,gene,start,type_g35
55141,ENST00000644773.1,TUG1,CTG,only


In [41]:
refseq_only_non_AUG[refseq_only_non_AUG['gene'] == 'TUG1']

,tr_id_refseq,gene,start,type_refseq


----

In [42]:
g35_both_df

,tr_id_g35,gene,start,type_g35
721,ENST00000374935.7,EIF4G3,ATG,both
722,ENST00000264211.12,EIF4G3,ATG,both
723,ENST00000602326.5,EIF4G3,ATG,both
724,ENST00000400422.6,EIF4G3,ATG,both
725,ENST00000634879.1,EIF4G3,GTG,both
...,...,...,...,...
55525,ENST00000652095.2,MRTFA,ATG,both
55526,ENST00000355630.9,MRTFA,GTG,both
55527,ENST00000402042.6,MRTFA,GTG,both
55528,ENST00000618417.1,MRTFA,ATG,both


In [43]:
# only non-AUG both in g35 and refseq
print ('%s both AUG and non-AUG both in g35 and refseq' % len(list(set(refseq_both.gene.unique()).intersection(set(g35_both_df.gene.unique())))))
print (', '.join(list(set(refseq_both.gene.unique()).intersection(set(g35_both_df.gene.unique())))))

print ('%s genes non-AUG in g35' % g35_both_df.gene.nunique())
print ('%s genes non-AUG in refseq' % refseq_both.gene.nunique())

13 both AUG and non-AUG both in g35 and refseq
BAG1, KCTD11, PEG10, HCK, OAZ3, FNDC5, WDR26, POU5F1, SWI5, MRTFA, WT1, TEAD4, NR1I2
30 genes non-AUG in g35
23 genes non-AUG in refseq


In [44]:
', '.join(list(set(g35_both_df.gene.unique()) - set(refseq_both.gene.unique()).intersection(set(g35_both_df.gene.unique()))))

'YPEL4, YPEL1, PIGX, R3HCC1, NFKBID, EIF4G3, POLG, IFT46, EFCAB14, YPEL2, SBK1, SLC22A17, MYC, MORN2, EED, TRAK1, UFSP1'

In [45]:
', '.join(list(set(refseq_both.gene.unique()) - set(refseq_both.gene.unique()).intersection(set(g35_both_df.gene.unique()))))

'IRAG1, PTEN, PANK2, PIM1, SP3, RARB, FGF2, MDFIC, VEGFA, CEBPA'

# Sanity check: PhyloSET and RiboSET overlaps 

In [46]:
ps = pd.read_csv('tmp_res/SET1.txt', sep='\t')
rb =  pd.read_csv('tmp_res/SET2.txt', sep='\t')

In [47]:
set(ps.gene.unique()).intersection(g35_both_df.gene.tolist()), set(ps.gene.unique()).intersection(g35_only_df.gene.tolist())

# OAZ3: extensions is different (diff transcripts, not overlapping, diff 5'UTRs, just gene name match)
# CACNG8: same transcript, CTG-init, ext is found in 5'UTR before annotated CTG (positive score)

({'OAZ3'}, {'CACNG8'})

In [48]:
set(ps.gene.unique()).intersection(refseq_both.gene.tolist()), set(ps.gene.unique()).intersection(refseq_only_non_AUG.gene.tolist())

({'OAZ3'}, {'CACNG8'})

In [49]:
set(rb.gene.unique()).intersection(g35_both_df.gene.tolist()), set(rb.gene.unique()).intersection(g35_only_df.gene.tolist())
# STIM2: same transcript, CTG-init, ext is found in 5'UTR before annotated CTG (positive score)

(set(), {'STIM2'})

In [50]:
set(rb.gene.unique()).intersection(refseq_both.gene.tolist()), set(rb.gene.unique()).intersection(refseq_only_non_AUG.gene.tolist())

(set(), {'STIM2'})

-----

# Compare gencode25/gencode35

In [51]:
metadata_pc_g25 = pd.read_csv('tmp_res/metadata_pc_g25.txt', sep='\t')


df_g25 = read_gtf("data/gencode.v25.annotation.gtf")
df_genes_g25 = df_g25[df_g25["feature"] == "gene"]
df_CDS_g25 = df_g25[df_g25["feature"] == "CDS"]

annotation_CDS_g25 = df_CDS_g25[['transcript_id', 'transcript_type', 'gene_name', 'tag']]
annotation_CDS_g25.columns = ['tr_id', 'transcript_type', 'gene', 'tag']

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


### Non-AUG tag

In [52]:
# we will create df with transcript type and tags 
li = []
                                                        
for i,feature in enumerate(DataIterator('data/gencode.v25.annotation.gtf')): 
    if i % 1000000 == 0:
        print (i)
    try:
        attr = feature.attributes
        gene_type = attr['gene_type'][0]
        transcript_id = attr['transcript_id'][0]
        gene_name = attr['gene_name'][0]
        tag = attr['tag']
        transcript_type = attr['transcript_type'][0]
            
        # cds_start_NF;cds_end_NF 
        if 'non_ATG_start' in tag:            
            li.append([transcript_id, gene_name, transcript_type, ';'.join(tag), 'non_ATG_start'])
        else:
            li.append([transcript_id, gene_name, transcript_type, ';'.join(tag), None])
            
    except Exception as e:
        continue           
    

0
1000000
2000000


In [53]:
df = pd.DataFrame(li, columns = ['tr_id', 'gene', 'biotype', 'tags', 'non-ATG-tag']).drop_duplicates()

# select genes with complete CDS, protein-coding transcript type and non-AUG tag 
non_AUG_annotated_pc_and_cds_complete_g25 = df[(df['non-ATG-tag'] == 'non_ATG_start') & (df['biotype'] == 'protein_coding') & 
                                                          (~df['tags'].str.contains('cds_start_NF')) &
                                                         (~df['tags'].str.contains('cds_end_NF'))]

genes_with_non_AUG_proteoforms_g25 = list(non_AUG_annotated_pc_and_cds_complete_g25['gene'].unique())

print (len(genes_with_non_AUG_proteoforms_g25))

24


In [54]:
cds_complete_and_pc_df_g25 = df[(df['biotype'] == 'protein_coding') & (~df['tags'].str.contains('cds_start_NF')) & (~df['tags'].str.contains('cds_end_NF'))]

In [55]:
# pc transcripts, complete cds -> merge with metadata
pr_trascript_cds_complete_df_g25 = cds_complete_and_pc_df_g25.merge(metadata_pc_g25, on=['gene', 'tr_id'], how='inner')

# select genes with non-AUG
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25 = pr_trascript_cds_complete_df_g25[pr_trascript_cds_complete_df_g25['gene'].isin(genes_with_non_AUG_proteoforms_g25)]

print (df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25.shape, df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25.gene.nunique(), Counter(df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25.cds_start_codon.tolist()))

(84, 17) 24 Counter({'ATG': 40, 'CTG': 34, 'TTG': 5, 'GTG': 3, 'ATA': 1, 'ACG': 1})


In [56]:
# CDS_start codon 
df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25['CDS_start'] = df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25['cds_seq'].str.slice(0, 3)

# gene - start codons dataframe
d1 = df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25.groupby('gene')['CDS_start'].apply(set).apply(list).apply(', '.join).to_dict()
gene_codon_g25_df = pd.DataFrame(zip(d1.keys(), d1.values()), columns = ['gene', 'codons'])

/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
print ('there are both AUG and non-AUG proteoforms for %s genes' % gene_codon_g25_df[(gene_codon_g25_df['codons'].str.contains('ATG')) & (gene_codon_g25_df['codons'].str.len() >= 6)].shape[0])
print (', '.join(gene_codon_g25_df[(gene_codon_g25_df['codons'].str.contains('ATG')) & (gene_codon_g25_df['codons'].str.len() >= 6)].gene.unique()))
print ()

print ('there are only AUG codon proteoform for %s genes (although its non-AUG annotated)' % gene_codon_g35_df[(gene_codon_g35_df['codons'] == 'ATG')].shape[0])
print (', '.join(gene_codon_g25_df[(gene_codon_g25_df['codons'] == 'ATG')].gene.unique()))
print ()

print ('there are only non-AUG codon proteoform for %s genes' % gene_codon_g25_df[(gene_codon_g25_df['codons'].str.len() <=3) & (gene_codon_g25_df['codons'] != 'ATG')].shape[0])
print (', '.join(gene_codon_g25_df[(gene_codon_g25_df['codons'].str.len() <=3) & (gene_codon_g25_df['codons'] != 'ATG')].gene.unique()))

there are both AUG and non-AUG proteoforms for 17 genes
BAG1, EIF4G3, FNDC5, HCK, MDFIC, MYC, NELFB, NR1I2, OAZ3, PEG10, PIGX, POU5F1, R3HCC1, SLC22A17, SWI5, TEAD4, WT1

there are only AUG codon proteoform for 1 genes (although its non-AUG annotated)


there are only non-AUG codon proteoform for 7 genes
CACNG8, GTF3A, NPTXR, RNF187, STIM2, TRPV6, TXNRD3


### both AUG/nonAUG or only nonAUG dfs

In [58]:
both_proteoforms_g25_genes = gene_codon_g25_df[(gene_codon_g25_df['codons'].str.contains('ATG')) & (gene_codon_g25_df['codons'].str.len() >= 6)].gene.unique()

only_non_AUG_proteoforms_genes_g25 = gene_codon_g25_df[(gene_codon_g25_df['codons'].str.len() <=3) & (gene_codon_g25_df['codons'] != 'ATG')].gene.unique()

print (len(only_non_AUG_proteoforms_genes_g25), len(both_proteoforms_g25_genes))

7 17


In [59]:
df = df_with_all_proteoforms_of_genes_with_non_AUG_complete_CDS_g25.copy()

g25_only_df = df[df['gene'].isin(only_non_AUG_proteoforms_genes_g25)][['tr_id', 'gene', 'cds_start_codon']]



g25_only_df2 = g25_only_df.merge(annotation_CDS_g25, on=['tr_id', 'gene'], how='inner').drop_duplicates()

g25_only_df3 = g25_only_df2[(g25_only_df2['transcript_type'] == 'protein_coding') 
            & (~g25_only_df2['tag'].str.contains('CDS_start_NF')) &
            (~g25_only_df2['tag'].str.contains('CDS_end_NF'))]

print (g25_only_df.gene.nunique(), g25_only_df3.gene.nunique())


g25_only_df3.to_csv('tmp_res/gencode25_only_non_AUG_PC_complete_CDS.txt', sep='\t', index=False)

7 7


In [60]:
g25_both_df = df[df['gene'].isin(both_proteoforms_g25_genes)][['tr_id', 'gene', 'cds_start_codon']]

g25_both_df2 = g25_both_df.merge(annotation_CDS_g25, on=['tr_id', 'gene'], how='inner').drop_duplicates()

g25_both_df3 = g25_both_df2[(g25_both_df2['transcript_type'] == 'protein_coding') 
            & (~g25_both_df2['tag'].str.contains('CDS_start_NF')) &
            (~g25_both_df2['tag'].str.contains('CDS_end_NF'))]

print (g25_both_df.gene.nunique(), g25_both_df3.gene.nunique())

g25_both_df3.to_csv('tmp_res/gencode25_both_AUG_non_AUG_PC_complete_CDS.txt', sep='\t', index=False)

17 17


---------------

In [61]:
g25_only_df['type_g25'] = 'only'
g25_only_df.columns = ['tr_id_g25', 'gene', 'start', 'type_g25']

g25_both_df['type_g25'] = 'both'
g25_both_df.columns = ['tr_id_g25', 'gene', 'start', 'type_g25']

In [62]:
# only non-AUG both in g35 and g25
print ('%s only non-AUG both in g25 and refseq' % len(list(set(g35_only_df.gene.unique()).intersection(set(g25_only_df.gene.unique())))))
print (', '.join(list(set(g35_only_df.gene.unique()).intersection(set(g25_only_df.gene.unique())))))

print ('%s genes non-AUG in g25' % g25_only_df.gene.nunique())
print ('%s genes non-AUG in g35' % g35_only_df.gene.nunique())

7 only non-AUG both in g25 and refseq
TRPV6, TXNRD3, GTF3A, NPTXR, STIM2, CACNG8, RNF187
7 genes non-AUG in g25
15 genes non-AUG in g35


In [63]:
# only non-AUG both in g35 and g25
print ('%s both AUG and non-AUG both in g35 and refseq' % len(list(set(g25_both_df.gene.unique()).intersection(set(g35_both_df.gene.unique())))))
print (', '.join(list(set(g25_both_df.gene.unique()).intersection(set(g35_both_df.gene.unique())))))

print ('%s genes non-AUG in g25' % g25_both_df.gene.nunique())
print ('%s genes non-AUG in g35' % g35_both_df.gene.nunique())

15 both AUG and non-AUG both in g35 and refseq
BAG1, PEG10, OAZ3, PIGX, R3HCC1, HCK, EIF4G3, FNDC5, POU5F1, SWI5, SLC22A17, MYC, TEAD4, WT1, NR1I2
17 genes non-AUG in g25
30 genes non-AUG in g35
